# Model Mimarisi: U-Net Generator ve PatchGAN Discriminator

Bu notebook, LDCT görüntü iyileştirme için kullandığımız hibrit Pix2Pix + WGAN-GP modelinin mimari tanımlarını içerir.

**İçindekiler:**
1. Encoder/Decoder blokları (downsample/upsample)
2. U-Net Generator
3. PatchGAN Discriminator
4. WGAN-GP Hybrit Model sınıfı

---

## Neden Bu Mimari?

Klasik Pix2Pix modelinde Binary Cross-Entropy loss kullanılırken, biz **Wasserstein loss + Gradient Penalty** kullanıyoruz. Bunun birkaç avantajı var:

- Eğitim daha stabil (mode collapse riski düşük)
- Discriminator'dan daha anlamlı gradyan akışı
- L1 loss ile birlikte kullanınca hem yapısal hem de piksel-düzeyinde benzerlik sağlanıyor

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

IMG_WIDTH = 256
IMG_HEIGHT = 256
CHANNELS = 1

## 1. Encoder ve Decoder Blokları

U-Net mimarisinin temel yapı taşları. Encoder bloğu görüntüyü küçültür ve özellik çıkarır, decoder bloğu ise geri büyütür.

- **downsample**: Conv2D → BatchNorm → LeakyReLU
- **upsample**: Conv2DTranspose → BatchNorm → Dropout (opsiyonel) → ReLU

In [ ]:
def downsample(filters, size, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = keras.Sequential()
    result.add(layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))
    if apply_batchnorm:
        result.add(layers.BatchNormalization())
    result.add(layers.LeakyReLU())
    return result

def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = keras.Sequential()
    result.add(layers.Conv2DTranspose(filters, size, strides=2, padding='same',
                                      kernel_initializer=initializer, use_bias=False))
    result.add(layers.BatchNormalization())
    if apply_dropout:
        result.add(layers.Dropout(0.5))
    result.add(layers.ReLU())
    return result

## 2. U-Net Generator

8 katmanlı encoder ve 7 katmanlı decoder. Skip connection'lar sayesinde düşük seviye özellikler korunuyor.

```
Input(256x256) → Encoder(8 blok) → Bottleneck(1x1) → Decoder(7 blok) → Output(256x256)
                      ↓                                    ↑
                      └──────── Skip Connections ──────────┘
```

Son katmanda `tanh` aktivasyonu kullanıyoruz çünkü çıktı [-1, 1] aralığında normalize edilmiş.

In [ ]:
def build_generator():
    inputs = layers.Input(shape=[IMG_WIDTH, IMG_HEIGHT, CHANNELS])

    # Encoder
    down_stack = [
        downsample(64, 4, apply_batchnorm=False), # (bs, 128, 128, 64)
        downsample(128, 4), # (bs, 64, 64, 128)
        downsample(256, 4), # (bs, 32, 32, 256)
        downsample(512, 4), # (bs, 16, 16, 512)
        downsample(512, 4), # (bs, 8, 8, 512)
        downsample(512, 4), # (bs, 4, 4, 512)
        downsample(512, 4), # (bs, 2, 2, 512)
        downsample(512, 4), # (bs, 1, 1, 512)
    ]

    # Decoder
    up_stack = [
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4),
        upsample(256, 4),
        upsample(128, 4),
        upsample(64, 4),
    ]

    initializer = tf.random_normal_initializer(0., 0.02)
    last = layers.Conv2DTranspose(CHANNELS, 4, strides=2, padding='same',
                                  kernel_initializer=initializer, activation='tanh')

    x = inputs
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)

    skips = reversed(skips[:-1])

    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = layers.Concatenate()([x, skip])

    x = last(x)
    return keras.Model(inputs=inputs, outputs=x)

## 3. PatchGAN Discriminator

70x70 patch üzerinde karar veren discriminator. Tüm görüntü yerine lokal bölgelere bakarak daha gerçekçi doku üretimini teşvik eder.

**Not:** WGAN-GP kullandığımız için son katmanda sigmoid yok. Çıktı doğrudan "Wasserstein distance" hesabında kullanılıyor.

In [ ]:
def build_discriminator():
    #   Sigmoid fonksiyonu yerine WGAN-GP kullanılmıştır.
    initializer = tf.random_normal_initializer(0., 0.02)

    inp = layers.Input(shape=[IMG_WIDTH, IMG_HEIGHT, CHANNELS], name='input_image')
    tar = layers.Input(shape=[IMG_WIDTH, IMG_HEIGHT, CHANNELS], name='target_image')

    x = layers.Concatenate()([inp, tar]) # (bs, 256, 256, channels*2)

    down1 = downsample(64, 4, False)(x)
    down2 = downsample(128, 4)(down1)
    down3 = downsample(256, 4)(down2)

    # Zero Padding ve Conv
    zero_pad1 = layers.ZeroPadding2D()(down3)
    conv = layers.Conv2D(512, 4, strides=1, kernel_initializer=initializer, use_bias=False)(zero_pad1)
    batchnorm1 = layers.BatchNormalization()(conv)
    leaky_relu = layers.LeakyReLU()(batchnorm1)

    zero_pad2 = layers.ZeroPadding2D()(leaky_relu)

    last = layers.Conv2D(1, 4, strides=1, kernel_initializer=initializer)(zero_pad2)

    return keras.Model(inputs=[inp, tar], outputs=last)

## 4. WGAN-GP + Pix2Pix Hibrit Model

Bu sınıf Keras Model API'sini kullanarak özel eğitim döngüsü tanımlar.

**Kayıp Fonksiyonları:**
- **Discriminator**: Wasserstein distance + Gradient Penalty
- **Generator**: Wasserstein loss + L1 Reconstruction loss

**Hiperparametreler:**
- `lambda_gp=10`: Gradient Penalty ağırlığı (WGAN-GP makalesinden)
- `lambda_l1=100`: L1 loss ağırlığı (orijinal Pix2Pix'ten)

In [ ]:
class WGAN_GP_Pix2Pix(keras.Model):
    def __init__(self, generator, discriminator, lambda_gp=10.0, lambda_l1=100.0):
        super(WGAN_GP_Pix2Pix, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.lambda_gp = lambda_gp # Gradient Penalty ağırlığı
        self.lambda_l1 = lambda_l1 # L1 (Pix2Pix) ağırlığı

    def compile(self, d_optimizer, g_optimizer):
        super(WGAN_GP_Pix2Pix, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_fn = self.wasserstein_loss
        self.g_loss_fn = self.wasserstein_loss
        self.l1_loss_fn = tf.keras.losses.MeanAbsoluteError()

    def wasserstein_loss(self, y_true, y_pred):
        return tf.reduce_mean(y_true * y_pred)

    def gradient_penalty(self, batch_size, real_images, fake_images, input_images):
        """ GP Hesaplama: Real ve Fake arası interpolasyon """
        alpha = tf.random.normal([batch_size, 1, 1, 1], 0.0, 1.0)
        diff = fake_images - real_images
        interpolated = real_images + alpha * diff

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            # Discriminator'a hem input(LD) hem interpolasyon verilir
            pred = self.discriminator([input_images, interpolated], training=True)

        grads = gp_tape.gradient(pred, [interpolated])[0]
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp

    def call(self, inputs, training=False):
        if isinstance(inputs, (list, tuple)):
            inputs = inputs[0]
        return self.generator(inputs, training=training)

    def train_step(self, data):
        # Data Loader'dan gelen veri: (input_image, target_image)
        input_image, target_image = data
        batch_size = tf.shape(input_image)[0]

        # --- DISCRIMINATOR EĞİTİMİ ---
        with tf.GradientTape() as tape:
            fake_image = self.generator(input_image, training=True)

            fake_pred = self.discriminator([input_image, fake_image], training=True)
            real_pred = self.discriminator([input_image, target_image], training=True)

            # Wasserstein Loss: D(fake) - D(real)

            # Not: Real için -1, Fake için 1 gibi davranılır, formül minimize etmek üzerine kuruludur.

            d_cost = tf.reduce_mean(fake_pred) - tf.reduce_mean(real_pred)

            # Gradient Penalty
            gp = self.gradient_penalty(batch_size, target_image, fake_image, input_image)

            # Toplam D Loss
            d_loss = d_cost + (gp * self.lambda_gp)

        d_grad = tape.gradient(d_loss, self.discriminator.trainable_variables)
        self.d_optimizer.apply_gradients(zip(d_grad, self.discriminator.trainable_variables))

        # --- GENERATOR EĞİTİMİ ---
        with tf.GradientTape() as tape:
            fake_image = self.generator(input_image, training=True)
            fake_pred = self.discriminator([input_image, fake_image], training=True)

            # G Loss (Wasserstein Kısmı)
            g_wgan_loss = -tf.reduce_mean(fake_pred)

            # G Loss (L1 Kısmı): Orijinal Pix2Pix yapısı (Görüntü benzerliği)
            g_l1_loss = self.l1_loss_fn(target_image, fake_image) * self.lambda_l1

            g_loss = g_wgan_loss + g_l1_loss

        g_grad = tape.gradient(g_loss, self.generator.trainable_variables)
        self.g_optimizer.apply_gradients(zip(g_grad, self.generator.trainable_variables))

        return {"d_loss": d_loss, "g_loss": g_loss, "g_l1": g_l1_loss}

## Model Oluşturma

Aşağıdaki hücreyi çalıştırarak modelleri oluşturabilirsiniz. Bu notebook'u diğer notebook'larda `%run` komutuyla çağırabilirsiniz.

In [ ]:
generator = build_generator()
discriminator = build_discriminator()

print(f"Generator parametreleri: {generator.count_params():,}")
print(f"Discriminator parametreleri: {discriminator.count_params():,}")